## Use of Tools for Agentic Workflow

In [47]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio



In [48]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [49]:
sales_agent1 = Agent(
        name="Professional_Sales_Agent",
        instructions=instructions1,
        model="gpt-4o-mini",
)

sales_agent2 = Agent(
        name="Engaging_Sales_Agent",
        instructions=instructions2,
        model="gpt-4o-mini",
)

sales_agent3 = Agent(
        name="Busy_Sales_Agent",
        instructions=instructions3,
        model="gpt-4o-mini",
)

In [50]:
sales_agent1

Agent(name='Professional_Sales_Agent', instructions='You are a sales agent working for ComplAI, a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. You write professional, serious cold emails.', prompt=None, handoff_description=None, handoffs=[], model='gpt-4o-mini', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool_choice=True)

In [51]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("toyeladun@gmail.com")  # Change to your verified sender
    to_email = To("toyeladun@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

Wrap agents in a tool -Method one - List out

In [52]:
#wrap the agents in a tool
message = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="salesagent1", tool_description="message")
tool2 = sales_agent2.as_tool(tool_name="salesagent2", tool_description="message")
tool3 = sales_agent3.as_tool(tool_name="salesagent3", tool_description="message")

Wrap agents in a tool -Method two -run a loop and store in a list

In [53]:
message = "Write a cold sales email"


allagents = [sales_agent1, sales_agent2, sales_agent3]

tools = [agent.as_tool(tool_name=agent.name, tool_description="message") for agent in allagents]


In [54]:
tools

[FunctionTool(name='Professional_Sales_Agent', description='message', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Professional_Sales_Agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002401A65B420>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='Engaging_Sales_Agent', description='message', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Engaging_Sales_Agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002401CA6A5C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='Busy_Sales_Agent', description='message', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 're

In [55]:
#append the sendgrid tool to the allagents list of tools making it four tools
#-3 email generation agents 
#-1 email sendgrid agent

In [56]:
tools.append(send_email)

In [57]:
tools

[FunctionTool(name='Professional_Sales_Agent', description='message', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Professional_Sales_Agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002401A65B420>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='Engaging_Sales_Agent', description='message', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'Engaging_Sales_Agent_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002401CA6A5C0>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='Busy_Sales_Agent', description='message', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 're

Lastly , we use the final agent to plan orchestration amongst these other agents/tools

In [58]:
managerinstructions = """
You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


planningagent = Agent(
    name = "Sales_Manager",
    instructions=managerinstructions,
    tools=tools,
    model='gpt-4o-mini'
)


with trace("Sales email picker"):
    result = await Runner.run(planningagent, "Send a cold email")


MaxTurnsExceeded: Max turns (10) exceeded